## 从文件中随机抽取15个样例

In [37]:
with open('/home/jinhanqi/tf_program/topic2essay_data/composition.txt', 'r') as f:
    con = f.readlines()
print len(con)

490000


In [38]:
print con[0]

地球 上 的 光明 和 温暖 ， 都 是 我 送来 的 。 没有 了 我 ， 地球 上 会 一片 凄凉 ， 没有 风 、 雪 、 雨 、 露 ， 也 没有 草 、 木 、 鸟 、 兽 ， 你们 人类 即使 生存 在 地球 上 也 不会 生存 多久 啊 ！ 



In [39]:
import random
random.shuffle(con)

In [40]:
print con[0]

表壳 里 ， 最长 的 针 是 秒针 ， 最短 的 是 时针 ， 而 那根 不长 也 不断 的 就是 分针 喽 ！ 时间 一分一秒 的 过去 了 。 俗话说 ： 一寸光阴一寸金 ， 寸金难买 寸 光阴 。 快快 戴 上 我 ， 让 我 告诉 你 ， 你 又 虚费 了 多少 光阴 ， 又 把握 了 多少 时间 吧 ！ 



In [41]:
samples = con[0:15]

In [42]:
articles = [s.split('</d>')[0] for s in samples]

In [43]:
print articles[0]

表壳 里 ， 最长 的 针 是 秒针 ， 最短 的 是 时针 ， 而 那根 不长 也 不断 的 就是 分针 喽 ！ 时间 一分一秒 的 过去 了 。 俗话说 ： 一寸光阴一寸金 ， 寸金难买 寸 光阴 。 快快 戴 上 我 ， 让 我 告诉 你 ， 你 又 虚费 了 多少 光阴 ， 又 把握 了 多少 时间 吧 ！ 



In [44]:
keywords = [s.split('</d>')[1] for s in samples]

IndexError: list index out of range

In [45]:
print keywords[0]

 头部 妈妈 千奇百怪 裂开 现在



## 定义TF-IDF模型

In [31]:
# -*- coding: utf-8 -*-
import math
import jieba
import jieba.posseg as psg
from gensim import corpora, models
from jieba import analyse
import functools
 
def get_stopword_list():
	stop_word_path = './/stopwords/stopword.txt'
	stopword_list = [sw.replace('\n','') for sw in open(stop_word_path).readlines()]
	return stopword_list
 
# 分词方法
def seg_to_list(sentence, pos=False):
	if not pos:
		# 不进行词性标注的分词方法
		seg_list = jieba.cut(sentence)
	else:
		# 进行词性标注的分词方法
		seg_list = psg.cut(sentence)
	return seg_list
 
# 去除干扰词，根据pos判断是否过滤除名词外的其他词性，再判断词是否在停用词表中，长度是否大于等于2等。
def word_filter(seg_list, pos=False):
	stopword_list = get_stopword_list()
	filter_list = []
	# 根据pos参数选择是否词性过滤
	# 不进行词性过滤，则将词性都标记为n,表示全部保留
	for seg in seg_list:
		if not pos:
			word = seg
			flag = 'n'
		else:
			word = seg.word
			flag = seg.flag
		if not flag.startswith('n'):
			continue
		# 过滤高停用词表中的词，以及长度为<2的词
		if not word in stopword_list and len(word)>1:
			filter_list.append(word)
	return filter_list
 
 # 数据加载
 
def load_data(pos=False,corpus_path='/home/jinhanqi/tf_program/topic2essay_data/null.txt'):
 	doc_list = []
 	for line in open(corpus_path, 'r'):
 		content = line.strip()
 		seg_list = seg_to_list(content, pos)
 		filter_list = word_filter(seg_list, pos)
 		doc_list.append(filter_list)
 	return doc_list
 
# idf值统计方法
def train_idf(doc_list):
	idf_dic = {}
	# 总文档数
	tt_count = len(doc_list)
	# 每个词出现的文档数
	for doc in doc_list:
		for word in set(doc):
			idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
 
	# 按公式转换为idf值，分母加1进行平滑处理
	for k, v in idf_dic.items():
		idf_dic[k] = math.log(tt_count/(1.0+v))
	# 对于没有在字典中的词，默认其尽在一个文档出现，得到默认idf值
	default_idf = math.log(tt_count/(1.0))
	return idf_dic, default_idf
 
# topK
def cmp(e1, e2):
	import numpy as np 
	res = np.sign(e1[1] - e2[1])
	if res != 0:
		return res 
	else:
		a = e1[0] + e2[0]
		b = e2[0] + e1[0]
		if a > b:
			return 1
		elif a == b:
			return 0
		else:
			return -1
 
# TF-IDF类
class TfIdf(object):
	# 训练好的idf字典，默认idf值，处理后的待提取文本，关键词数量
	def __init__(self, idf_dic, default_idf, word_list, keyword_num):
		self.word_list = word_list
		self.idf_dic, self.default_idf = idf_dic, default_idf
		self.tf_dic = self.get_tf_dic()
		self.keyword_num = keyword_num
	#统计tf值
	def get_tf_dic(self):
		tf_dic = {}
		for word in self.word_list:
			tf_dic[word] = tf_dic.get(word, 0.0) + 1.0
		tt_count = len(self.word_list)
		for k,v in tf_dic.items():
			tf_dic[k] = float(v)/tt_count
		return tf_dic
	# 按公式计算tf-idf
	def get_tfidf(self):
		tfidf_dic = {}
		for word in self.word_list:
			idf = self.idf_dic.get(word, self.default_idf)
			tf = self.tf_dic.get(word, 0)
			tfidf = tf*idf
			tfidf_dic[word] = tfidf
		res = [] # 根据tf-idf排序，取排名前keyword_num的词作为关键词
        
		for k,v in sorted(tfidf_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
			res.append(k)
		return res
 
# 主题模型
class TopicModel(object):
	# 
	def __init__(self, doc_list, keyword_num, model="LSI", num_topics=4):
		# 使用gensim接口，将文本转为向量化表示
		self.dictionary = corpora.Dictionary(doc_list)
		# 使用BOW模型向量化
		corpus = [self.dictionary.doc2bow(doc) for doc in doc_list]
		# 对每个词，根据tf-idf进行加权，得到加权后的向量表示
		self.tfidf_model = models.TfidfModel(corpus)
		self.corpus_tfidf = self.tfidf_model[corpus]
		
		self.keyword_num = keyword_num
		self.num_topics = num_topics
		# 选择加载的模型
		if model == 'LSI':
			self.model = self.train_lsi()
		else:
			self.model = self.train_lda()
		# 得到数据集的主题-词分布
		word_dic = self.word_dictionary(doc_list)
		self.wordtopic_dic = self.get_wordtopic(word_dic)
 
	def train_lsi(self):
		lsi = models.LsiModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
		return lsi
 
	def train_lda(self):
		lda = models.LdaModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
		return lda
 
	def get_wordtopic(self, word_dic):
		wordtopic_dic = {}
		for word in word_dic:
			single_list = [word]
			wordcorpus = self.tfidf_model[self.dictionary.doc2bow(single_list)]
			wordtopic = self.model[wordcorpus]
			wordtopic_dic[word] = wordtopic
		return wordtopic_dic
 
    # 词空间构建方法和向量化方法，在没有gensim接口时的一般处理方法
	def word_dictionary(self, doc_list):
		dictionary = []
		for doc in doc_list:
			dictionary.extend(doc)
 
		dictionary = list(set(dictionary))
 
		return dictionary
 
	def doc2bowvec(self, word_list):
		vec_list = [1 if word in word_list else 0 for word in self.dictionary]
		return vec_list
 
	# 计算词的分布和文档的分布的相似度，取相似度最高的keyword_num个词作为关键词
	def get_simword(self, word_list):
		sentcorpus = self.tfidf_model[self.dictionary.doc2bow(word_list)]
		senttopic = self.model[sentcorpus]
		# 余弦相似度计算
		def calsim(l1, l2):
			a, b, c = 0.0, 0.0, 0.0
			for t1, t2 in zip(l1, l2):
				x1 = t1[1]
				x2 = t2[1]
				a += x1 * x1
				b += x1 * x1
				c += x2 * x2
			sim = a / math.sqrt(b * c) if not (b * c) == 0.0 else 0.0
			return sim
		# 计算输入文本和每个词的主题分布相似度
		sim_dic = {}
		for k,v in self.wordtopic_dic.items():
			if k not in word_list:
				continue
			sim = calsim(v, senttopic)
			sim_dic[k] = sim
 
		for k,v in sorted(sim_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
			print(k)
		print ""
 
 
def tfidf_extract(word_list, pos=False, keyword_num=10):
	doc_list = load_data(pos)
	idf_dic, default_idf = train_idf(doc_list)
	tfidf_model = TfIdf(idf_dic, default_idf, word_list, keyword_num)
	return tfidf_model.get_tfidf()
 
def textrank_extract(text, pos=False, keyword_num=10):
	textrank = analyse.textrank
	keywords = textrank(text, keyword_num)
	# 输出抽取出的关键词
	for keyword in keywords:
		print(keyword)
	#print()
 
def topic_extract(word_list, model, pos=False, keyword_num=10):
	doc_list = load_data(pos)
	topic_model = TopicModel(doc_list, keyword_num, model=model)
	topic_model.get_simword(word_list)
    
def getTFIDF(text):
    pos = True
    seg_list = seg_to_list(text, pos)
    filter_list = word_filter(seg_list, pos)
    return tfidf_extract(filter_list)

## 得到关键词

In [32]:
keywords_TF = [getTFIDF(text) for text in articles]

/home/jinhanqi/.local/lib/python2.7/site-packages/ipykernel_launcher.py:40: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [33]:
def printChinese(text):
    print json.dumps(text, ensure_ascii = False, encoding = 'utf-8')

In [34]:
printChinese(keywords_TF[0])

["部位", "妈妈", "头部"]


In [35]:
for i in range(len(articles)):
    print articles[i]
    print keywords[i]
    printChinese(keywords_TF[i])
    print ''
    print ''

可 这些 我 虽然 画 得 很 好 ， 可 我 一直 想 画 一个 人 。 尤其 是 我 辛辛苦苦 的 妈妈 。 但是 ， 现在 ， 我 只 会 画 头部 ， 其它 部位 画 得 千奇百怪 。 一些 斜 着 ， 一些 还 裂开 了 ， 画 不成 妈妈 。 
 头部 妈妈 千奇百怪 裂开 现在

["部位", "妈妈", "头部"]


听 了 妈妈 的话 ， 小云 放开 了 手中 的 菊花 ， 不好意思 地 说 ： “ 对不起 ， 妈妈 ， 我 知道 错 了 。 ” 妈妈 摸 着 小云 的 头 说 ： “ 我 的 好孩子 ， 我们 一起 做 个 爱护 环境 的 人 。 ” 
 妈妈 爱护 环境 知道 对不起

["菊花", "环境", "小云", "妈妈", "好孩子"]


小 池塘 的 旁边 还有 许多 小山 和 大树 ， 茂密 的 树叶 好像 搭起 了 一个个 凉棚 。 小 池塘 清澈见底 ， 就连 在 水底 的 细小 沙粒 和 青 褐色 的 石头 都 看 得 一清二楚 。 水底 有 许多 小 金鱼 和 小蝌蚪 ， 每个 小蝌蚪 都 像 一个个 黑色 的 小 逗号 。 
 好像 褐色 细小 池塘 金鱼

["黑色", "金鱼", "逗号", "褐色", "细小", "石头", "沙粒", "池塘", "水底", "树叶"]


边检 员 做事 要 谨慎 ， 认真 ， 镇静 ， 反应 要 快 ， 这些 余翔 都 具备 了 。 看 完 这部 电影 ， 我 在 想 ： 世界 上 还有 什么 事 能比 边检 要 更加 谨慎 ， 认真 ， 镇静 ， 反应 要 快 的 。 此时此刻 我 对 他们 肃然起敬 。 
 此时此刻 电影 世界 做事 反应

["电影", "余翔", "世界"]


“ 我 来 ！ ” 我 说 。 “ 啊 ？ 你 ？ 哼 ， 你 骗人 吧 ！ 女生 没有 一个 爬 得 上去 的 ！ 我们 劝 你 还是 放弃 吧 ！ ” 听 着 那 刺激 我 内心 的 话语 ， 我 深深 的 伤心 了 ， 当 我 准备 放弃 时 ， 我 想到 了 父亲 曾 告诫 我 的话 ： 完成 一件 事 ， 就要 坚持到底 ， 不能 半途而废 。 于是 ， 我 饱含 着 伤人 的 话语 ， 走到 了 单杠 面前 。 
 放弃 话语 不能 想到 准备

["骗人", "话语", 